In [ ]:
import tensorflow
import keras

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/CatClassifier/train'

In [ ]:
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=15,)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227,227),
    batch_size=32,
    subset='training',
    class_mode='binary',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227,227),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)


Found 20001 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# 추가 학습을 위해 미리 저장한 모델을 불러옴
additional_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/AlexNet')

In [ ]:
additional_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_5 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)      

In [ ]:
additional_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay = 0.0005),
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/CatClassifier/AlexNet',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',)

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose = 1, mode= 'auto')

In [ ]:
num_train_samples = train_generator.n
num_train_batches = train_generator.batch_size
print(num_train_samples,' , ',num_train_batches)
train_size = num_train_samples // num_train_batches
print(train_size)

20001  ,  32
625


In [ ]:
num_val_samples = validation_generator.n
num_val_batches = validation_generator.batch_size
print(num_val_samples,' , ',num_val_batches)
val_size = num_val_samples // num_val_batches
print(val_size)

5000  ,  32
156


In [ ]:
hist = additional_model.fit(train_generator,
                 steps_per_epoch=train_size, 
                 epochs = 100 , 
                 validation_data=validation_generator ,
                 validation_steps=val_size ,
                 verbose=1,
                 callbacks=[checkpoint, early])

Epoch 1/100
625/625 [==============================] - 15968s 26s/step - loss: 0.4299 - accuracy: 0.8045 - val_loss: 0.6174 - val_accuracy: 0.6867

Epoch 00001: val_accuracy improved from -inf to 0.68670, saving model to /content/drive/MyDrive/CatClassifier/AlexNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/AlexNet/assets
Epoch 2/100
625/625 [==============================] - 380s 608ms/step - loss: 0.4094 - accuracy: 0.8140 - val_loss: 0.6853 - val_accuracy: 0.6550

Epoch 00002: val_accuracy did not improve from 0.68670
Epoch 3/100
625/625 [==============================] - 378s 605ms/step - loss: 0.3633 - accuracy: 0.8350 - val_loss: 0.3668 - val_accuracy: 0.8359

Epoch 00003: val_accuracy improved from 0.68670 to 0.83594, saving model to /content/drive/MyDrive/CatClassifier/AlexNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/AlexNet/assets
Epoch 4/100
625/625 [==============================] - 368s 590ms/step - loss: 0.3440 - 

In [ ]:
additional_model.evaluate(train_generator)

626/626 [==============================] - 304s 485ms/step - loss: 0.2151 - accuracy: 0.9095


[0.21505282819271088, 0.9095045328140259]

In [ ]:
additional_model.save('/content/drive/MyDrive/CatClassifier/AlexNet/additional train (2)')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/AlexNet/additional train (2)/assets
